# Eseguire script come processo della pipeline

Una pipeline consente di raggruppare più passaggi in un unico flusso di lavoro. È possibile creare una pipeline con componenti. Ogni componente riflette uno script Python da eseguire. Un componente viene definito in un file YAML che specifica lo script e come eseguirlo. 

## Prima di iniziare

Per eseguire il codice in questo notebook, è necessaria la versione più recente del pacchetto  **azureml-ai-ml** . Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguire `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Dopo aver installato i pacchetti SDK necessari, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati automaticamente. Per completare il comando è necessario solo l'ID sottoscrizione.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro in alto a destra di Studio. Verrà aperto un riquadro a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Creare gli script

Si creerà una pipeline con due passaggi:

1. **Preparare i dati**: correggere i dati mancanti e normalizzare i dati.
1. **Eseguire il training del modello**: esegue il training di un modello di classificazione albero delle decisioni.

Eseguire le celle seguenti per creare la cartella **src** e i due script.

In [ ]:
## Definire i componenti

Per definire il componente che è necessario specificare:

- **Metadati**: *nome*, *nome visualizzato*, *versione*, *descrizione*, *tipo* e così via. I metadati consentono di descrivere e gestire il componente.
- **Interfaccia**: *input* e *output*. Ad esempio, un componente di training del modello accetta i dati di training e la frequenza di regolarizzazione come input e genera un file di modello sottoposto a training come output. 
- **Comando, codice & ambiente**: *il comando*, il *codice* e *l'ambiente* per eseguire il componente. Comando è il comando shell per eseguire il componente. Il codice si riferisce in genere a una directory del codice sorgente. L'ambiente può essere un ambiente AzureML (curato o creato in modo personalizzato), un'immagine Docker o un ambiente conda.

Eseguire le celle seguenti per creare un FILE YAML per ogni componente che si vuole eseguire come passaggio della pipeline.

In [ ]:
## Caricare i componenti

Dopo aver definito ogni componente, è possibile caricare i componenti facendo riferimento ai file YAML. 

## Creare la pipeline

Dopo aver creato e caricato i componenti, è possibile compilare la pipeline. I due componenti verranno creati in una pipeline. In primo luogo, è necessario eseguire il `prep_data` componente. L'output del primo componente deve essere l'input del secondo componente `train_decision_tree`, che eseguirà il training del modello.

La `diabetes_classification` funzione rappresenta la pipeline completa. La funzione prevede una variabile di input: `pipeline_job_input`. Un asset di dati è stato creato durante l'installazione. Si userà l'asset di dati registrato come input della pipeline. 

In [ ]:
È possibile recuperare la configurazione del processo della pipeline stampando l'oggetto `pipeline_job` :

In [ ]:
È possibile modificare qualsiasi parametro della configurazione del processo della pipeline facendo riferimento al parametro e specificando il nuovo valore:

In [ ]:
## Inviare il processo della pipeline

Infine, dopo aver compilato la pipeline e configurato il processo della pipeline per l'esecuzione come richiesto, è possibile inviare il processo della pipeline:

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job